# Dilithium

## Description

<p>CRYSTALS-Dilithium is a digital signature scheme. It replaces signature schemes based on factorization and elliptic curves.</p>

## Math model

### Signing

<p>
Given a message m, the signing process involves the following steps:
1. Generate a nonce r using a deterministic function based on the message m and a secret key sk.
2. Compute a vector z and a hint h using the nonce r, the secret key sk, and the message m.
3. The signature σ is composed of the vector z and the hint h.
</p>

### Verifying

<p>
Given a signature σ and a message m, the verifying process involves the following steps:
1. Parse the signature σ into the vector z and the hint h.
2. Use the public key pk, the message m, and the hint h to compute a value w.
3. Check if the vector z and the value w satisfy certain conditions. If they do, the signature is valid; otherwise, it is invalid.
</p>


In [112]:
import numpy as np
from hashlib import sha256

# Helper function to hash a message and map it to a polynomial
def hash_to_poly(message, n, q):
    hash_bytes = sha256(message).digest()
    hash_int = int.from_bytes(hash_bytes, byteorder='big')
    poly = np.zeros(n, dtype=int)
    for i in range(n):
        poly[i] = (hash_int >> (i % 32)) & 1  # Map hash bits to polynomial coefficients
    return poly % q

# Helper function to generate random polynomials with small coefficients
def random_small_poly(n):
    return np.random.randint(-1, 2, n)  # Coefficients in {-1, 0, 1}

# Helper function for polynomial multiplication in Z_q[X]/(X^n + 1)
def poly_mult(a, b, q, n):
    result = np.zeros(n, dtype=int)
    for i in range(n):
        for j in range(n):
            idx = (i + j) % n
            result[idx] = (result[idx] + a[i] * b[j]) % q
            if i + j >= n:
                result[idx] = (result[idx] - a[i] * b[j]) % q
    return result

# Key Generation
def key_gen(n, q):
    # Private key: small random polynomials
    s1 = random_small_poly(n)
    s2 = random_small_poly(n)
    
    # Public key: A (random matrix) and t = A * s1 + s2
    A = random_small_poly(n)
    t = poly_mult(A, s1, q, n)
    t = (t + s2) % q
    
    return (A, t), (s1, s2)

# Signing
def sign(message, private_key, n, q):
    s1, s2 = private_key
    
    # Hash the message to create a challenge polynomial
    c = hash_to_poly(message, n, q)
    
    # Compute the signature: z = s1 * c + s2
    z = poly_mult(s1, c, q, n)
    z = (z + s2) % q
    
    return z

# Verification
def verify(message, signature, public_key, n, q):
    A, t = public_key
    
    # Hash the message to recreate the challenge polynomial
    c = hash_to_poly(message, n, q)
    
    # Compute A * z
    Az = poly_mult(A, signature, q, n)
    
    # Compute t * c
    tc = poly_mult(t, c, q, n)
    
    # Check if Az is close to tc
    diff = (Az - tc) % q
    norm = np.linalg.norm(diff)
    
    # Threshold for verification
    threshold = 10  # Adjusted for small coefficients
    return norm < threshold

if __name__ == "__main__":
    # Parameters
    n = 16  # Degree of polynomials
    q = 17  # Modulus

    # Key generation
    public_key, private_key = key_gen(n, q)

    # Message to sign
    message = b"Hello, Dilithium!"

    # Sign the message
    signature = sign(message, private_key, n, q)

    # Verify the signature
    is_valid = verify(message, signature, public_key, n, q)

    print("Signature valid:", is_valid)
    print("Public key:", public_key)
    print("Signature:", signature)

Signature valid: False
Public key: (array([ 1,  1,  0,  0,  0, -1,  1,  1, -1,  0,  0,  0,  0,  0,  0,  0]), array([16,  0,  0, 16,  0,  2, 14, 15, 16, 15, 16,  2,  1, 15,  1,  1]))
Signature: [16  0 16 16 16  0 14 15 13 15 15  0 15 15  1 15]
